In [29]:
import sys
sys.path.append('../../')
import time
import argparse
import os
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from utils.pytorchtools import EarlyStopping
from utils.data import load_data
from utils.focalloss import FocalLoss, DSCLoss
from sklearn.metrics import confusion_matrix

In [30]:
feats_type = 1
features_list, adjM, labels, train_val_test_idx, dl = load_data("ERM_E")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

##### 测试集评估

In [31]:
y_true = dl.labels_test['data'][dl.labels_test['mask']].argmax(axis=1)
y_pred = pd.read_table("./ERM_N_1.txt",header=None)[3].values

In [32]:
# Calculate the classification report
report = classification_report(y_true, y_pred, zero_division=0, output_dict=True)
print(pd.DataFrame(report))
# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
# Print the confusion matrix
print(conf_matrix)

                    0          1  accuracy   macro avg  weighted avg
precision    0.991755   0.967213   0.99011    0.979484      0.989975
recall       0.997630   0.893939   0.99011    0.945785      0.990110
f1-score     0.994684   0.929134   0.99011    0.961909      0.989930
support    844.000000  66.000000   0.99011  910.000000    910.000000
[[842   2]
 [  7  59]]


#### 原始结点

##### 转录因子识别错误

In [33]:
output = pd.read_table(f"./ERM_E_1.txt",header=None)
label = pd.read_table('../../data/ERM/label.dat.test',header=None)
label = label.sort_values(0)
output.columns = ['Idx','Ept','GroudTruth','Predict']
output['GroudTruth'] = label[3].values
output = output[['Idx','GroudTruth','Predict']]
test_node = pd.read_csv("/home/linjw/GNNs/HGB/NC/benchmark/data/ERMer/test_node.csv")
train_node = pd.read_csv("/home/linjw/GNNs/HGB/NC/benchmark/data/ERMer/train_node.csv")
output

,Idx,GroudTruth,Predict
0,3,0,0
1,4,0,0
2,6,0,0
3,10,0,0
4,13,0,0
...,...,...,...
905,3012,0,0
906,3018,0,0
907,3031,0,0
908,3032,0,0


In [34]:
target = output[(output['GroudTruth']!=1) & (output['Predict']==1)]
out = test_node[test_node['nid'].isin(target.Idx.values)]
out

,id,amino_acid_seq:String,~label,label,nid
497,b3472,MRNLVKYVGIGLLVMGLAACDDKDTNATAQGSVAESNATGNPVNLL...,0,Gene,2286
829,b2667,MTELAQLQASAEQAAALLKAMSHPKRLLILCMLSGSPGTSAGELTR...,0,Gene,1698


In [35]:
test_fasta_df = test_node[["id","amino_acid_seq:String"]]
train_fasta_df = train_node[["id","amino_acid_seq:String"]]
test_fasta_df

,id,amino_acid_seq:String
0,b0023,MANIKSAKKRAIQSEKARKHNASRRSMMRTFIKKVYAAIEAGDKAA...
1,b1783,MNIFDHYRQRYEAAKDEEFTLQEFLTTCRQDRSAYANAAERLLMAI...
2,b0192,MVKKAIVTAMAVISLFTLMGCNNRAEVDTLSPAQAAELKPMPQSWR...
3,b1386,MGSPSLYSARKTTLALAVALSFAWQAPVFAHGGEAHMVPMDKTLKE...
4,b0463,MNKNRGFTPLAVVLMLSGSLALTGCDDKQAQQGGQQMPAVGVVTVK...
...,...,...
905,b1501,MKKKIESYQGAAGGWGAVKSVANAVRKQMDIRQDVIAMFDMNKPEG...
906,b1328,MKREEIADLMAFVVVAEERSFTRAAARLSMAQSALSQIVRRIEERL...
907,b4216,MTLRKILALTCLLLPMMASAHQFETGQRVPPIGITDRGELVLDKDQ...
908,b3811,MTDLHTDVERYLRYLSVERQLSPITLLNYQRQLEAIINFASENGLQ...


##### fasta文件

In [36]:
# 指定保存FASTA文件的路径
fasta_file_path = 'output/deeptfpredcit/test.fasta'

# 打开文件并将DataFrame写入FASTA文件
with open(fasta_file_path, 'w') as fasta_file:
    for index, row in test_fasta_df.iterrows():
        header = '>' + row['id'] + '\n'
        sequence = row['amino_acid_seq:String'] + '\n'
        fasta_file.write(header)
        fasta_file.write(sequence)

print(f'FASTA文件已保存到: {fasta_file_path}')

FASTA文件已保存到: output/deeptfpredcit/test.fasta


In [37]:
# 指定保存FASTA文件的路径
fasta_file_path = 'output/deeptfpredcit/train.fasta'

# 打开文件并将DataFrame写入FASTA文件
with open(fasta_file_path, 'w') as fasta_file:
    for index, row in train_fasta_df.iterrows():
        header = '>' + row['id'] + '\n'
        sequence = row['amino_acid_seq:String'] + '\n'
        fasta_file.write(header)
        fasta_file.write(sequence)

print(f'FASTA文件已保存到: {fasta_file_path}')

FASTA文件已保存到: output/deeptfpredcit/train.fasta


In [38]:
###
# makeblastdb -in 数据库.fa -dbtype prot
# blastp -query test.fasta -subject train.fasta -out output.txt -outfmt 6

In [39]:
#makeblastdb -in train.fasta -dbtype prot

SyntaxError: invalid syntax (677876866.py, line 1)

##### DeepTFPredict

In [9]:
# !cd /home/linjw/BioML/deeptfactor 
# !python tf_running.py -i output/deeptfpredcit/test.fasta -o output/deeptfpredcit -g cpu

In [10]:
deeptfpredcit = pd.read_table(f"./output/deeptfpredcit/prediction_result.txt")
deeptfpredcit

,sequence_ID,prediction,score
0,b0023,False,0.0045
1,b1783,False,0.0096
2,b0192,False,0.0075
3,b1386,False,0.0074
4,b0463,False,0.0046
...,...,...,...
895,b1501,False,0.2175
896,b1328,True,0.9973
897,b4216,False,0.0099
898,b3811,False,0.0089


In [11]:
dp_pd = pd.merge(test_node,deeptfpredcit,left_on='id', right_on='sequence_ID')
dp_pd

,id,amino_acid_seq:String,~label,label,nid,sequence_ID,prediction,score
0,b0023,MANIKSAKKRAIQSEKARKHNASRRSMMRTFIKKVYAAIEAGDKAA...,0,Gene,6,b0023,False,0.0045
1,b1783,MNIFDHYRQRYEAAKDEEFTLQEFLTTCRQDRSAYANAAERLLMAI...,0,Gene,1171,b1783,False,0.0096
2,b0192,MVKKAIVTAMAVISLFTLMGCNNRAEVDTLSPAQAAELKPMPQSWR...,0,Gene,99,b0192,False,0.0075
3,b1386,MGSPSLYSARKTTLALAVALSFAWQAPVFAHGGEAHMVPMDKTLKE...,0,Gene,927,b1386,False,0.0074
4,b0463,MNKNRGFTPLAVVLMLSGSLALTGCDDKQAQQGGQQMPAVGVVTVK...,0,Gene,314,b0463,False,0.0046
...,...,...,...,...,...,...,...,...
895,b1501,MKKKIESYQGAAGGWGAVKSVANAVRKQMDIRQDVIAMFDMNKPEG...,0,Gene,968,b1501,False,0.2175
896,b1328,MKREEIADLMAFVVVAEERSFTRAAARLSMAQSALSQIVRRIEERL...,1,Gene,877,b1328,True,0.9973
897,b4216,MTLRKILALTCLLLPMMASAHQFETGQRVPPIGITDRGELVLDKDQ...,0,Gene,2834,b4216,False,0.0099
898,b3811,MTDLHTDVERYLRYLSVERQLSPITLLNYQRQLEAIINFASENGLQ...,0,Gene,2545,b3811,False,0.0089


In [12]:
dp_report = pd.DataFrame(classification_report(dp_pd["~label"].values, dp_pd["prediction"].values, zero_division=0, output_dict=True))
dp_report

,0,1,accuracy,macro avg,weighted avg
precision,0.996346,0.797468,0.978889,0.896907,0.981762
recall,0.980815,0.954545,0.978889,0.967680,0.978889
f1-score,0.988520,0.868966,0.978889,0.928743,0.979752
support,834.000000,66.000000,0.978889,900.000000,900.000000
